In [3]:
import duckdb
import pandas as pd 
from pathlib import Path

# handling the path, i show where the database and sql file is 
duckdb_path = "../data/sakila.duckdb"
sql_load_path = "../sql/load_sakila.sql"

# Cleans old datafile if it exists, so thar we start a new every time
Path(duckdb_path).unlink(missing_ok = True)

# here i use the with statement to make sure the connection is closed after use
# i also  do the connection and the readiing of the sql file in the same line as Debbie did
# to ensure both are properly closed after use
with duckdb.connect(duckdb_path) as conn, open(sql_load_path, 'r') as ingest_script:
    # this runs the sql script that connects to the file 
    conn.execute(ingest_script.read())

    # This will import tables to pandas
    # also when the connection is open we can run sql queries, and also convert them to Pandas Data frames
    print("load the tables into memory")
    actor = conn.sql("SELECT * FROM actor").df()
    category = conn.sql("SELECT * FROM category").df()
    film = conn.sql("SELECT * FROM film").df()
    film_actor = conn.sql("SELECT * FROM film_actor").df()
    film_category = conn.sql("SELECT * FROM film_category").df()

#this verifies that the data has been loaded correctly by printing the first 5 rows of each table
# so that i cn ensure myself that the data has been loaded correctly
print(" data has been loaded correctly,")

print(f"testing it out. First movie: {film['title'][0]}")






load the tables into memory
 data has been loaded correctly,
testing it out. First movie: ACADEMY DINOSAUR


In [4]:
# task 1a, 
print(" This is presenting the anwser: Movies longer than 3 hours")

# here i use duckdb to query the data directly for the variable film
# my choice of the where to be in minutes (180 minutes = 3 hours), is beacuse the length column is in minutes
movie_3h_plus = duckdb.sql("""
    SELECT title, length
    FROM film
    WHERE length > 180
    ORDER BY length DESC
""").df()

print(movie_3h_plus)
    

 This is presenting the anwser: Movies longer than 3 hours
                 title  length
0         WORST BANGER     185
1        CHICAGO NORTH     185
2       CONTROL ANTHEM     185
3       DARN FORRESTER     185
4    SWEET BROTHERHOOD     185
5          GANGS PRIDE     185
6            HOME PITY     185
7   SOLDIERS EVOLUTION     185
8         POND SEATTLE     185
9        MUSCLE BRIGHT     185
10     MOONWALKER FOOL     184
11      THEORY MERMAID     184
12   CONSPIRACY SPIRIT     184
13    CRYSTAL BREAKING     184
14      SORORITY QUEEN     184
15      SONS INTERVIEW     184
16     SMOOCHY CONTROL     184
17      KING EVOLUTION     184
18      YOUNG LANGUAGE     183
19       SCALAWAG DUCK     183
20      FRONTIER CABIN     183
21       CATCH AMISTAD     183
22           WIFE TURN     183
23       MONSOON CAUSE     182
24       RECORDS ZORRO     182
25          REDS POCUS     182
26         SATURN NAME     182
27      SEARCHERS WAIT     182
28     BAKED CLEOPATRA     182
29       LO

In [5]:
#task 1b,
print(" This is presenting the movies that have the word love in title, also showing these columns: title, rating, length, description")

movie_title_love = duckdb.sql("""
    SELECT title, rating, length, description
    FROM film 
    WHERE title LIKE '%LOVE%'
    """).df()
# instead of just printing the results i use the display function to show the results in a more readable way
# Note: that LLM helped me with figuring out the display function, and now it looks much better 
display(movie_title_love)




 This is presenting the movies that have the word love in title, also showing these columns: title, rating, length, description


,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


In [ ]:
# task 1b, 
# this is a nother way of doing this task, but here i only want to dispaly hte exakt word love!

print("This is presenting the movies that have the exact word love in title, also showing these columns: title, rating, length, description")

movie_title_love_exact = duckdb.sql("""
                                                                    
    SELECT title, rating, length, description
    FROM film
    WHERE title LIKE 'LOVE %'
    OR title LIKE '% LOVE %'
    OR title LIKE '% LOVE'
    OR title = 'LOVE'
    

""").df()
# in the query above i have used multiple conditions to ensure that only the exact word love is matched in the title
# the or functions help to cover different positions of the word love in the title
display(movie_title_love_exact)


This is presenting the movies that have the exact word love in title, also showing these columns: title, rating, length, description


,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
3,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
4,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...


In [ ]:
# task 1c,
print(" This is presenting the the calculation on the length column, showing the shortest, longest and average movie length")
# here i i calculate the minimum , maximum and average length of the movies
length_calc_movie = duckdb.sql("""
    SELECT 
        MIN(length) as short_movie_stats,
        MAX(length) as long_movie_stats,
        AVG(length) as avg_movie_stats
    FROM film
    """).df()
# here again i use the display function for a more readble output
# i even was able to round the average value with the round function
display(round(length_calc_movie))

 This is presenting the the calculation on the length column, showing the shortest, longest and average movie length


,short_movie_stats,long_movie_stats,avg_movie_stats
0,46,185,115.0


In [ ]:
# task 1d,
print(" This will represent the Top 10 most expensive movies to rent per day (rental Rates)")
# here what i did os select all needed columns,
# then i divieded the rental rate with the rental duration, 
# As the alias cost_per_day
most_expensive_movies = duckdb.sql("""
    SELECT  
    title,
    rental_rate,
    rental_duration,
    (rental_rate / rental_duration) as cost_per_day
FROM film 
ORDER BY cost_per_day DESC
LIMIT 10

                                   
    
     """).df()

display(most_expensive_movies)

 This will represent the Top 10 most expensive movies to rent per day (rental Rates)


,title,rental_rate,rental_duration,cost_per_day
0,BACKLASH UNDEFEATED,4.99,3,1.663333
1,BILKO ANONYMOUS,4.99,3,1.663333
2,BEAST HUNCHBACK,4.99,3,1.663333
3,AUTUMN CROW,4.99,3,1.663333
4,ACE GOLDFINGER,4.99,3,1.663333
5,CARIBBEAN LIBERTY,4.99,3,1.663333
6,BEHAVIOR RUNAWAY,4.99,3,1.663333
7,CASPER DRAGONFLY,4.99,3,1.663333
8,AMERICAN CIRCUS,4.99,3,1.663333
9,CASUALTIES ENCINO,4.99,3,1.663333


In [19]:
# Task 1e
print(" This represents the actors that have played in most movies, and is displaying the top 10")
#  get the first name and last name from the actor table. And i give it tha alias (a)
# i then did a JOIN, and this is so that i put together the right actor with the right movie via the ID. 
# What i then did is use GROUP BY so i can count the amount per person.

actors_most_movies = duckdb.sql("""
SELECT 
    a.first_name,
    a.last_name,
    COUNT(fa.film_id) as movie_counter
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY movie_counter DESC
LIMIT 10
""").df()

display(actors_most_movies)

 This represents the actors that have played in most movies, and is displaying the top 10


,first_name,last_name,movie_counter
0,GINA,DEGENERES,42
1,WALTER,TORN,41
2,MARY,KEITEL,40
3,MATTHEW,CARREY,39
4,SANDRA,KILMER,37
5,SCARLETT,DAMON,36
6,HENRY,BERRY,35
7,VIVIEN,BASINGER,35
8,ANGELA,WITHERSPOON,35
9,GROUCHO,DUNST,35


## 1f 
### In this part i will be woking on my own questions
1. What movies generates the most money in total?
2. What movie has the most copies in stock?
3. What months had the highedt sales?
4.  what rating has the most rental price 